In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
dfs = pd.DataFrame({})

for i in range(4,10):
    df = pd.read_csv('./trento/sms-call-internet-tn-2013-11-04.csv'.format(i), parse_dates=['time'])
    dfs = dfs.append(df)
#df = pd.read_csv('./csv/sms-call-internet-tn-2013-11-10.csv', parse_dates=['time'])
#dfs = dfs.append(df)
dfs = dfs.fillna(0)
del df

In [3]:
#Group by hours and aggregate values according to each cell/hour
dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-4)
dfgg = dfgr[['hour', 'cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour', 'cellid'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
del dfs
print(dfgg)

#Group by hours and aggregate values according to the hour
#dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
#dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-1)
#dfgg = dfgr[['hour', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
#print(dfgg)

        hour  cellid      smsin     smsout    callin   callout    internet
0          0      38   5.969678   6.232408  2.807108  0.635891  427.081829
1          0      39   7.548749   2.214684  0.849466  0.689014  386.361336
2          0      40  19.041941   9.035045  2.099748  0.360504  432.430146
3          0     154   9.267153   3.394019  0.000000  1.697009  378.473690
4          0     155  12.436047   2.280616  0.745892  1.057392  555.647800
...      ...     ...        ...        ...       ...       ...         ...
149922    23   11336   3.495751   0.395651  0.047594  1.529881   48.642250
149923    23   11337   8.029184   5.061059  0.288979  1.967369  191.183256
149924    23   11452  62.951238  57.828120  0.967373  5.674942  856.066510
149925    23   11453  17.327898  10.755993  0.982905  2.493540  330.453343
149926    23   11454   5.487752   4.259755  0.704577  1.493912  410.465780

[149927 rows x 7 columns]


In [4]:
l=[]
#6134 - 3911
for i in range(3911, 6135, 117):
    for j in range(i, i+20):
        l.append(j)
#print(l)

In [5]:
df20 = pd.DataFrame({})
for el in dfgg.iterrows():
    if(l.count(int(el[1].cellid))==1):
        tmp = pd.DataFrame([[(el[1].hour), int(el[1].cellid), el[1].smsin, el[1].smsout, el[1].callin, el[1].callout, el[1].internet]], columns=['hour', 'cellid', 'smsin','smsout', 'callin','callout', 'internet'])
        df20=df20.append(tmp)

In [6]:
df20 = df20.set_index(['hour']).sort_index()
print(df20)

      cellid      smsin     smsout     callin   callout     internet
hour                                                                
0.0     3911  61.976381  71.825485  10.565011  3.953341   796.193818
0.0     3912  76.944103  50.963924   7.440468  4.345746   883.944422
0.0     3913  56.172129  88.186281  15.746384  2.631150  1112.644050
0.0     3914  28.013164  42.094594   8.003901  3.844593   915.406552
0.0     3915  28.743422  21.540295   1.639229  1.404649  1099.999593
...      ...        ...        ...        ...       ...          ...
23.0    6149  31.097134  30.482175   5.222796  3.506093   410.017056
23.0    6150  16.211609  15.452421   2.094752  0.879405   279.995464
23.0    6151   9.734368  13.035468   1.366937  0.049753   243.813995
23.0    6152   4.726861   6.964128   0.540441  0.075502   129.326308
23.0    6153   1.696294   1.735022   0.230893  0.192657    26.464192

[9600 rows x 6 columns]


In [7]:
cells = df20.cellid
scaler=MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df20), columns=df20.columns, index=df20.index)

In [8]:
dfgg=scaled
dfgg['cellid']=cells
print(dfgg)

      cellid     smsin    smsout    callin   callout  internet
hour                                                          
0.0     3911  0.003200  0.003938  0.001495  0.000462  0.006921
0.0     3912  0.003972  0.002794  0.001053  0.000508  0.007703
0.0     3913  0.002900  0.004835  0.002229  0.000307  0.009742
0.0     3914  0.001446  0.002308  0.001133  0.000449  0.007984
0.0     3915  0.001484  0.001181  0.000232  0.000164  0.009630
...      ...       ...       ...       ...       ...       ...
23.0    6149  0.001605  0.001671  0.000739  0.000410  0.003478
23.0    6150  0.000837  0.000847  0.000297  0.000103  0.002319
23.0    6151  0.000503  0.000715  0.000193  0.000006  0.001996
23.0    6152  0.000244  0.000382  0.000076  0.000009  0.000976
23.0    6153  0.000088  0.000095  0.000033  0.000023  0.000059

[9600 rows x 6 columns]


In [9]:
#dfgg.to_csv('./classif/20x20/tn/tntemp.csv')

In [10]:
dfsi = [] #data from df without indexing
dfsi2 = pd.DataFrame({}) #new df to save
dfso = []
dfso2 = pd.DataFrame({})
dfci = []
dfci2 = pd.DataFrame({})
dfco = []
dfco2 = pd.DataFrame({})
dfin = []
dfin2 = pd.DataFrame({})

dfsi.append(dfgg.loc[0].cellid)
dfso.append(dfgg.loc[0].cellid)
dfci.append(dfgg.loc[0].cellid)
dfco.append(dfgg.loc[0].cellid)
dfin.append(dfgg.loc[0].cellid)

dfsi2["cellid"]=dfsi[0].values
dfso2["cellid"]=dfsi[0].values
dfci2["cellid"]=dfsi[0].values
dfco2["cellid"]=dfsi[0].values
dfin2["cellid"]=dfsi[0].values
for i in range(0, dfgr.hour.max()+1): #dfgr.hour.max()+1
    #row[0] - hour, row[1]: df cols
    dfsi.append(dfgg.loc[i].smsin)
    dfsi2["smsin"+str(i)]=dfsi[i+1].values
    
    dfso.append(dfgg.loc[i].smsout)
    dfso2["smsout"+str(i)]=dfso[i+1].values
    
    dfci.append(dfgg.loc[i].callin)
    dfci2["callin"+str(i)]=dfci[i+1].values
    
    dfco.append(dfgg.loc[i].callout)
    dfco2["callout"+str(i)]=dfco[i+1].values
    
    dfin.append(dfgg.loc[i].internet)
    dfin2["internet"+str(i)]=dfin[i+1].values


In [11]:
dfsi2.to_csv("./classif/20x20/tn/classif20x20week_smsin.csv")
dfsi2.head()

,cellid,smsin0,smsin1,smsin2,smsin3,smsin4,smsin5,smsin6,smsin7,smsin8,...,smsin14,smsin15,smsin16,smsin17,smsin18,smsin19,smsin20,smsin21,smsin22,smsin23
0,3911,0.003200,0.000691,0.000000,0.000878,0.000898,0.000366,0.000858,0.006003,0.008359,...,0.011202,0.010427,0.011257,0.012710,0.008667,0.009063,0.010357,0.010128,0.004532,0.005057
1,3912,0.003972,0.000596,0.000175,0.000892,0.001066,0.000434,0.001154,0.005800,0.008968,...,0.011135,0.008369,0.011626,0.011628,0.009229,0.009539,0.010899,0.013784,0.007777,0.004507
2,3913,0.002900,0.000241,0.000717,0.000086,0.000135,0.000409,0.001952,0.005594,0.014482,...,0.014979,0.016333,0.017792,0.016676,0.016649,0.014262,0.015437,0.016502,0.010103,0.006204
3,3914,0.001446,0.000538,0.000499,0.000174,0.000098,0.000443,0.001490,0.005812,0.012428,...,0.013435,0.013426,0.012991,0.012771,0.011130,0.011940,0.011569,0.010664,0.007076,0.003730
4,3915,0.001484,0.000429,0.000445,0.000017,0.000268,0.002128,0.000646,0.007579,0.015603,...,0.014163,0.018759,0.015171,0.017038,0.009778,0.012637,0.008012,0.007538,0.006157,0.002858


In [12]:
dfso2.to_csv("./classif/20x20/tn/classif20x20week_smsout.csv")
dfso2.head()

,cellid,smsout0,smsout1,smsout2,smsout3,smsout4,smsout5,smsout6,smsout7,smsout8,...,smsout14,smsout15,smsout16,smsout17,smsout18,smsout19,smsout20,smsout21,smsout22,smsout23
0,3911,0.003938,0.002909,0.002363,0.002284,0.002233,0.001676,0.001025,0.002610,0.004961,...,0.005874,0.006392,0.006161,0.011327,0.008606,0.006855,0.008754,0.007469,0.003526,0.003661
1,3912,0.002794,0.000791,0.000275,0.000000,0.000000,0.000232,0.001140,0.002200,0.003128,...,0.005106,0.003995,0.006208,0.010668,0.008105,0.007794,0.008065,0.008086,0.005680,0.004344
2,3913,0.004835,0.001541,0.001285,0.001122,0.001116,0.000731,0.001176,0.003794,0.007780,...,0.008478,0.007443,0.010664,0.011747,0.013843,0.012283,0.012366,0.011444,0.010540,0.005641
3,3914,0.002308,0.000320,0.000583,0.000493,0.000608,0.000237,0.002078,0.005019,0.005881,...,0.008189,0.008255,0.007090,0.007288,0.008622,0.008159,0.008595,0.010064,0.006686,0.003997
4,3915,0.001181,0.000144,0.000146,0.000133,0.000121,0.000088,0.001296,0.005669,0.005483,...,0.007333,0.009114,0.006413,0.011729,0.007731,0.007989,0.004740,0.006863,0.005745,0.003037


In [13]:
dfci2.to_csv("./classif/20x20/tn/classif20x20week_callin.csv")
dfci2.head()

,cellid,callin0,callin1,callin2,callin3,callin4,callin5,callin6,callin7,callin8,...,callin14,callin15,callin16,callin17,callin18,callin19,callin20,callin21,callin22,callin23
0,3911,0.001495,0.000430,0.000205,0.000187,0.000000,0.000374,0.001464,0.007137,0.018797,...,0.019244,0.019158,0.023355,0.026129,0.023040,0.014969,0.009492,0.006116,0.002322,0.001837
1,3912,0.001053,0.000233,0.000000,0.000235,0.000031,0.000210,0.000584,0.005925,0.015543,...,0.016837,0.016011,0.020362,0.023245,0.021010,0.013565,0.006195,0.005836,0.000916,0.000882
2,3913,0.002229,0.000690,0.000333,0.000356,0.000285,0.000592,0.001263,0.008427,0.025502,...,0.028709,0.029630,0.030236,0.029922,0.027468,0.020740,0.014959,0.010217,0.003061,0.003008
3,3914,0.001133,0.000980,0.000446,0.000408,0.000488,0.000969,0.001719,0.008498,0.017772,...,0.020722,0.024692,0.029017,0.026311,0.023713,0.016419,0.013990,0.007874,0.003730,0.002107
4,3915,0.000232,0.000548,0.001097,0.000029,0.000199,0.000252,0.000801,0.007368,0.017177,...,0.023571,0.028487,0.028978,0.031622,0.022491,0.016393,0.010363,0.005627,0.002978,0.001922


In [14]:
dfco2.to_csv("./classif/20x20/tn/classif20x20week_callout.csv")
dfco2.head()

,cellid,callout0,callout1,callout2,callout3,callout4,callout5,callout6,callout7,callout8,...,callout14,callout15,callout16,callout17,callout18,callout19,callout20,callout21,callout22,callout23
0,3911,0.000462,0.000338,0.000139,0.000199,0.000139,0.000030,0.001039,0.008887,0.020144,...,0.016434,0.020608,0.026198,0.025946,0.017393,0.012738,0.010467,0.007428,0.002187,0.002705
1,3912,0.000508,0.000338,0.000000,0.000005,0.000000,0.000000,0.001217,0.007904,0.014577,...,0.013812,0.014863,0.020062,0.019671,0.017635,0.012003,0.009502,0.006503,0.001139,0.001081
2,3913,0.000307,0.000019,0.000068,0.000129,0.000068,0.000030,0.003456,0.010757,0.020586,...,0.023117,0.023277,0.029671,0.025969,0.031934,0.015540,0.012870,0.008423,0.003945,0.001572
3,3914,0.000449,0.000129,0.000085,0.000095,0.000060,0.000775,0.004408,0.010271,0.018764,...,0.021840,0.022930,0.026407,0.027359,0.023296,0.016112,0.011351,0.005754,0.002936,0.000987
4,3915,0.000164,0.000488,0.000644,0.000374,0.000015,0.000455,0.007449,0.007354,0.018002,...,0.026799,0.030748,0.027171,0.025362,0.023042,0.015630,0.010471,0.006392,0.003256,0.002061


In [15]:
dfin2.to_csv("./classif/20x20/tn/classif20x20week_internet.csv")
dfin2.head()

,cellid,internet0,internet1,internet2,internet3,internet4,internet5,internet6,internet7,internet8,...,internet14,internet15,internet16,internet17,internet18,internet19,internet20,internet21,internet22,internet23
0,3911,0.006921,0.005433,0.004857,0.004618,0.004772,0.005006,0.007262,0.013781,0.015206,...,0.011188,0.011416,0.013215,0.013627,0.012408,0.010821,0.011349,0.008517,0.007580,0.006487
1,3912,0.007703,0.006018,0.005043,0.005333,0.005869,0.006052,0.007484,0.012729,0.014805,...,0.012093,0.011164,0.011941,0.012832,0.012313,0.012676,0.011999,0.010179,0.009462,0.009287
2,3913,0.009742,0.008222,0.007118,0.007861,0.009175,0.009618,0.011596,0.016689,0.021201,...,0.019294,0.017430,0.016584,0.016815,0.018100,0.020087,0.017045,0.014798,0.014319,0.014620
3,3914,0.007984,0.007022,0.005924,0.006140,0.006893,0.007384,0.010102,0.013316,0.015581,...,0.015640,0.014927,0.015246,0.016120,0.016333,0.016362,0.014764,0.013208,0.011657,0.010203
4,3915,0.009630,0.009105,0.007728,0.007269,0.007469,0.007689,0.009061,0.010981,0.018671,...,0.020194,0.021424,0.020630,0.019137,0.017223,0.019309,0.015293,0.015217,0.014120,0.008875
